<a href="https://colab.research.google.com/github/amarparab28/Dy.Tech/blob/main/Final_G42.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving Ses01M_script01_1.txt to Ses01M_script01_1.txt


In [3]:
import pandas as pd
import re

# Load the dataset
file_path = 'Ses01M_script01_1.txt'
data = []

with open(file_path, 'r') as file:
    lines = file.readlines()
    current_entry = {}
    for line in lines:
        # Check for start and end time lines
        match = re.match(r'\[(\d+\.\d+) - (\d+\.\d+)\]\s+(\S+)\s+(\w+)\s+\[(\d+\.\d+), (\d+\.\d+), (\d+\.\d+)\]', line)
        if match:
            if current_entry:
                data.append(current_entry)
            current_entry = {
                'start_time': float(match.group(1)),
                'end_time': float(match.group(2)),
                'turn_name': match.group(3),
                'emotion': match.group(4),
                'valence': float(match.group(5)),
                'arousal': float(match.group(6)),
                'dominance': float(match.group(7)),
                'annotations': []
            }
        else:
            # Check for annotation lines
            annotation_match = re.match(r'(\w+-E\d+):\s+([\w\s]+);\s*\(\)', line)
            if annotation_match and current_entry:
                current_entry['annotations'].append((annotation_match.group(1), annotation_match.group(2)))

    if current_entry:
        data.append(current_entry)

df = pd.DataFrame(data)


In [4]:
# Inspect the first few rows of the dataframe
print(df.head())

# Check for missing values
print(df.isnull().sum())

# Fill or drop missing values if any
df.fillna('', inplace=True)


   start_time  end_time               turn_name emotion  valence  arousal  \
0      8.0600   11.5914  Ses01M_script01_1_F000     fru   2.0000   1.6667   
1     14.4000   16.6181  Ses01M_script01_1_F001     sur   2.0000   2.6667   
2     19.2815   23.9200  Ses01M_script01_1_F002     sur   2.0000   3.3333   
3     29.3349   33.2000  Ses01M_script01_1_F003     sur   2.6667   2.6667   
4     39.7634   42.1267  Ses01M_script01_1_F004     sad   2.0000   2.0000   

   dominance                                        annotations  
0     1.6667  [(C-E1, Frustration), (C-E2, Sadness), (C-E4, ...  
1     3.0000  [(C-E1, Anger), (C-E2, Surprise), (C-E4, Surpr...  
2     3.0000  [(C-E1, Frustration), (C-E2, Surprise), (C-E4,...  
3     2.6667  [(C-E1, Frustration), (C-E2, Surprise), (C-E4,...  
4     2.0000   [(C-E1, Fear), (C-E2, Sadness), (C-E4, Sadness)]  
start_time     0
end_time       0
turn_name      0
emotion        0
valence        0
arousal        0
dominance      0
annotations    0
dtype

In [5]:
# Convert annotations to a single string for simplicity
df['annotations'] = df['annotations'].apply(lambda x: ' '.join([f'{k}:{v}' for k, v in x]))

# Encode categorical variables
emotion_mapping = {emotion: idx for idx, emotion in enumerate(df['emotion'].unique())}
df['emotion'] = df['emotion'].map(emotion_mapping)

# Normalize VAD scores (optional, but might be useful for model training)
df['valence'] = df['valence'] / df['valence'].max()
df['arousal'] = df['arousal'] / df['arousal'].max()
df['dominance'] = df['dominance'] / df['dominance'].max()

# Combine all relevant features into a single string for the transformer model input
df['input_text'] = df.apply(lambda row: f"{row['turn_name']} {row['annotations']} Emotion:{row['emotion']} VAD:[{row['valence']}, {row['arousal']}, {row['dominance']}]", axis=1)

# Print the processed DataFrame
print(df[['start_time', 'end_time', 'input_text']].head())


   start_time  end_time                                         input_text
0      8.0600   11.5914  Ses01M_script01_1_F000 C-E1:Frustration C-E2:S...
1     14.4000   16.6181  Ses01M_script01_1_F001 C-E1:Anger C-E2:Surpris...
2     19.2815   23.9200  Ses01M_script01_1_F002 C-E1:Frustration C-E2:S...
3     29.3349   33.2000  Ses01M_script01_1_F003 C-E1:Frustration C-E2:S...
4     39.7634   42.1267  Ses01M_script01_1_F004 C-E1:Fear C-E2:Sadness ...


In [6]:
from transformers import BertTokenizer, BertModel

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the input text
inputs = tokenizer(df['input_text'].tolist(), return_tensors='pt', padding=True, truncation=True)

# Print the tokenized inputs
print(inputs)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': tensor([[ 101, 7367, 2015,  ...,    0,    0,    0],
        [ 101, 7367, 2015,  ...,    0,    0,    0],
        [ 101, 7367, 2015,  ...,    0,    0,    0],
        ...,
        [ 101, 7367, 2015,  ...,    0,    0,    0],
        [ 101, 7367, 2015,  ...,    0,    0,    0],
        [ 101, 7367, 2015,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [7]:
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Define the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(emotion_mapping))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Prepare dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

labels = df['emotion'].tolist()
dataset = CustomDataset(inputs, labels)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset
)

# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-7-5f624d8efe47>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss


Step,Training Loss


TrainOutput(global_step=66, training_loss=1.7591274145877722, metrics={'train_runtime': 302.0652, 'train_samples_per_second': 0.864, 'train_steps_per_second': 0.218, 'total_flos': 11401032030660.0, 'train_loss': 1.7591274145877722, 'epoch': 3.0})

In [8]:
import numpy as np
from sklearn.metrics import mean_absolute_error
from transformers import EvalPrediction

# Define a function to compute metrics
def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Compute loss
    loss = np.mean(pred.predictions)

    # Compute MAE
    mae = mean_absolute_error(labels, preds)

    return {"loss": loss, "mae": mae}

# Modify training arguments to include evaluation steps
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",  # Evaluate every epoch
)

# Define Trainer with compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer.train()

# Evaluate the model on the training dataset (for demonstration purposes)
results = trainer.evaluate()

print(f"Final evaluation results: {results}")


<ipython-input-7-5f624d8efe47>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Mae
1,No log,-0.105431,1.356322
2,No log,-0.044081,0.505747
3,No log,0.018444,0.436782


<ipython-input-7-5f624d8efe47>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-7-5f624d8efe47>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-7-5f624d8efe47>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-7-5f624d8efe47>:25: UserWarning: To copy construct from a 

Final evaluation results: {'eval_loss': 0.0184437595307827, 'eval_mae': 0.4367816091954023, 'eval_runtime': 21.6838, 'eval_samples_per_second': 4.012, 'eval_steps_per_second': 1.015, 'epoch': 3.0}


In [11]:
import pandas as pd
import re
import numpy as np
import torch
from sklearn.metrics import mean_absolute_error
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EvalPrediction

# Load and process dataset
file_path = 'Ses01M_script01_1.txt'
data = []

with open(file_path, 'r') as file:
    lines = file.readlines()
    current_entry = {}
    for line in lines:
        match = re.match(r'\[(\d+\.\d+) - (\d+\.\d+)\]\s+(\S+)\s+(\w+)\s+\[(\d+\.\d+), (\d+\.\d+), (\d+\.\d+)\]', line)
        if match:
            if current_entry:
                data.append(current_entry)
            current_entry = {
                'start_time': float(match.group(1)),
                'end_time': float(match.group(2)),
                'turn_name': match.group(3),
                'emotion': match.group(4),
                'valence': float(match.group(5)),
                'arousal': float(match.group(6)),
                'dominance': float(match.group(7)),
                'annotations': []
            }
        else:
            annotation_match = re.match(r'(\w+-E\d+):\s+([\w\s]+);\s*\(\)', line)
            if annotation_match and current_entry:
                current_entry['annotations'].append((annotation_match.group(1), annotation_match.group(2)))

    if current_entry:
        data.append(current_entry)

df = pd.DataFrame(data)

# Data cleaning
df.fillna('', inplace=True)
df['annotations'] = df['annotations'].apply(lambda x: ' '.join([f'{k}:{v}' for k, v in x]))

# Encode categorical variables
emotion_mapping = {emotion: idx for idx, emotion in enumerate(df['emotion'].unique())}
df['emotion'] = df['emotion'].map(emotion_mapping)

# Normalize VAD scores (optional)
df['valence'] = df['valence'] / df['valence'].max()
df['arousal'] = df['arousal'] / df['arousal'].max()
df['dominance'] = df['dominance'] / df['dominance'].max()

# Combine all relevant features into a single string for the transformer model input
df['input_text'] = df.apply(lambda row: f"{row['turn_name']} {row['annotations']} Emotion:{row['emotion']} VAD:[{row['valence']}, {row['arousal']}, {row['dominance']}]", axis=1)

# Tokenize the input text
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
inputs = tokenizer(df['input_text'].tolist(), return_tensors='pt', padding=True, truncation=True)

# Define a function to compute metrics
def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Compute loss
    loss = np.mean(pred.predictions)

    # Compute MAE
    mae = mean_absolute_error(labels, preds)

    return {"loss": loss, "mae": mae}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",  # Evaluate every epoch
)

# Define dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

labels = df['emotion'].tolist()
dataset = CustomDataset(inputs, labels)

# Define Trainer with compute_metrics function
trainer = Trainer(
    model=BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(emotion_mapping)),
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model on the training dataset (for demonstration purposes)
results = trainer.evaluate()

print(f"Final evaluation results: {results}")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-11-86fbf4f37555>:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Mae
1,No log,-0.209362,1.379310
2,No log,-0.206614,2.310345
3,No log,-0.215921,1.367816


<ipython-input-11-86fbf4f37555>:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-86fbf4f37555>:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-86fbf4f37555>:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-86fbf4f37555>:92: UserWarning: To copy construct fro

Final evaluation results: {'eval_loss': -0.21592050790786743, 'eval_mae': 1.367816091954023, 'eval_runtime': 23.8699, 'eval_samples_per_second': 3.645, 'eval_steps_per_second': 0.922, 'epoch': 3.0}
